##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [4]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = genai.GenerativeModel(model_name="gemini-1.5-flash-exp-0827")
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IndoYXQgaXMgY2hpa24gbnVnZ2l0In1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiQ2hpa24gTnVnZ2l0IGlzIGFuIGFuaW1hdGVkIHdlYiBzZXJpZXMgY3JlYXRlZCBieSBLeXJhIEt1cGV0c2t5LCBmZWF0dXJpbmcgYSBxdWlya3ksIGxvbmctZWFyZWQgeWVsbG93IGRvZyBvZiB0aGUgc2FtZSBuYW1lIGFuZCBoaXMgZnJpZW5kcywgd2hvIGFsc28gaGF2ZSBmYXN0LWZvb2QtaW5zcGlyZWQgbmFtZXMuICBUaGUgc2VyaWVzIGlzIGtub3duIGZvciBpdHMgY29tZWRpYywgb2Z0ZW4gc3VycmVhbCwgYW5kIHNvbWV0aW1lcyBzcG9va3kgYWR2ZW50dXJlcy4gIFRoZSBlcGlzb2RlcyBhcmUgc2hvcnQsIHR5cGljYWxseSBsYXN0aW5nIGJldHdlZW4gMTUgYW5kIDIwIHNlY29uZHMgZWFjaC5cXG5cXG5UaGUgc2VyaWVzIHByZW1pZXJlZCBpbiBPY3RvYmVyIDIwMjAgb24gcGxhdGZvcm1zIGxpa2UgVGlrVG9rLCBYIChmb3JtZXJseSBUd2l0dGVyKSwgSW5zdGFncmFtLCBhbmQgbGF0ZXIgWW91VHViZS4gIEl0IHF1aWNrbHkgZ2FpbmVkIHBvcHVsYXJpdHksIGFtYXNzaW5nIG92ZXIgOCBtaWxsaW9uIGZvbGxvd2VycyBhbmQgYSBiaWxsaW9uIHZpZXdzIGFjcm9zcyB0aGVzZSBwbGF0Zm9ybXMgYXMgb2YgMjAyMy4gQ2hpa24gTnVnZ2l0IGhhcyBhbHNvIGV4cGFuZGVkIGludG8gbWVyY2hhbmRpc2UuICBBIGxvbmdlci1mb3JtIGFuaW1hdGVkIHNlcmllcyBpcyBjdXJyZW50bHkgaW4gZGV2ZWxvcG1lbnQgYXMgb2YgSnVuZSAyMDIzLCBhcyBhIGNvbGxhYm9yYXRpb24gYmV0d2VlbiBCdXp6RmVlZCBBbmltYXRpb24gTGFiIGFuZCBGcmVkZXJhdG9yIE5ldHdvcmsuXFxuXFxuQmV5b25kIHRoZSBodW1vciwgQ2hpa24gTnVnZ2l0IGFsc28gaW5jb3Jwb3JhdGVzIHRoZW1lcyBvZiBpbmNsdXNpdml0eSBhbmQgc2VsZi1hY2NlcHRhbmNlLCB3aXRoIG1hbnkgY2hhcmFjdGVycyBpZGVudGlmeWluZyBhcyBMR0JUUSsuICBTZXZlcmFsIGVwaXNvZGVzIGV4cGxvcmUgc3RvcnlsaW5lcyByZWxhdGVkIHRvIHBlcnNvbmFsIGlkZW50aXR5IGFuZCByZWxhdGlvbnNoaXBzLiBUaGUgbWFpbiBjaGFyYWN0ZXIsIENoaWtuIE51Z2dpdCwgaXMgZXZlbnR1YWxseSByZXZlYWxlZCB0byBiZSB0aGUgRGVtaWdvZCBvZiBDaGFvcywgZGVzdGluZWQgdG8gYnJlYWsgdGhlIGZhYnJpYyBvZiByZWFsaXR5LCBhIGRlc3RpbnkgaGUgdWx0aW1hdGVseSByZWplY3RzLlxcbiJ9XX0seyJyb2xlIjoidXNlciIsInBhcnRzIjpbeyJ0ZXh0IjoiZGVzY3JpYmUgSXNjcmVhbSBmcm9tIHRoZSBzZXJpZXMgQ2hpa24gTnVnZ2l0In1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiSXNjcmVhbSwgdm9pY2VkIGJ5IE5hdGUgQ2hhcnBlbnRpZXIsIGlzIGEgY2hhcmFjdGVyIGluIHRoZSBDaGlrbiBOdWdnaXQgc2VyaWVzLiAgVGhleSBhcHBlYXIgYXMgYW4gYWRvcmFibGUsIHBpbmsgYW5kIHdoaXRlIHJhYmJpdCBidXQgYXJlIHNlY3JldGx5IGEgdGVycmlmeWluZywgZ2VuZGVybGVzcyBkZW1vbi4gIFRoZWlyIHJlYWwgbmFtZSBpc24ndCBhY3R1YWxseSBcXFwiSXNjcmVhbSxcXFwiIGFuZCB0aGV5IHJldmVsIGluIGNhdXNpbmcgY2hhb3MuIElzY3JlYW0gaXMgRndlbmNoIEZ3eSdzIHBhcnRuZXIuXFxuIn1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJleHBhbmQgdXBvbiB0aGlzIGRlc2NyaXB0aW9uIG9mIElzY3JlYW0ifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJJc2NyZWFtIGlzIGEgbWFpbiBjaGFyYWN0ZXIgaW4gdGhlICpDaGlrbiBOdWdnaXQqIHdlYiBzZXJpZXMsIHZvaWNlZCBieSBOYXRlIENoYXJwZW50aWVyLiBXaGlsZSBhcHBlYXJpbmcgYXMgYSBjdXRlLCBwaW5rIGFuZCB3aGl0ZSBidW5ueSByYWJiaXQgd2l0aCB1bnNldHRsaW5nbHkgbGFyZ2UsIHJlZCBleWVzLCBJc2NyZWFtIGlzIHNlY3JldGx5IGEgdGVycmlmeWluZywgZ2VuZGVybGVzcyBkZW1vbiB3aG9zZSB0cnVlIG5hbWUgaXMgdW5rbm93bi4gVGhleSBkZWxpZ2h0IGluIHNvd2luZyBjaGFvcyBhbmQgYXJlIGtub3duIGZvciB0aGVpciBtaXNjaGlldm91cyBuYXR1cmUuICBEZXNwaXRlIHRoZWlyIGRlbW9uaWMgbmF0dXJlIGFuZCBsb3ZlIG9mIG1heWhlbSwgSXNjcmVhbSBpcyBwb3J0cmF5ZWQgYXMgZ2VudWluZWx5IGZyaWVuZGx5IGFuZCBpcyBpbiBhIHJlbGF0aW9uc2hpcCB3aXRoIGFub3RoZXIgbWFpbiBjaGFyYWN0ZXIsIEZ3ZW5jaCBGd3kuICBUaGV5IGFyZSBvZnRlbiBzZWVuIHRvZ2V0aGVyLCBhZGRpbmcgYW5vdGhlciBsYXllciBvZiBpbnRyaWd1ZSB0byB0aGVpciBkZW1vbmljIHBlcnNvbmEuXFxuIn1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJkZXNjcmliZSBob3cgSXNjcmVhbSBtYXkgaW50ZXJhY3Qgd2l0aCBGd2VuY2ggRnd5In1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiSXNjcmVhbSBhbmQgRndlbmNoIEZ3eSdzIGludGVyYWN0aW9ucyBhcmUgYSBtaXggb2YgY2hhb3RpYyBlbmVyZ3kgYW5kIHN1cnByaXNpbmdseSBzd2VldCBtb21lbnRzLiAgWW91J2xsIG9mdGVuIHNlZSB0aGVtIGVuZ2FnaW5nIGluOlxcblxcbiogKipNaXNjaGllZiBhbmQgUHJhbmtzOioqICBUaGV5IGxvdmUgdG8gdGVhbSB1cCBmb3IgcHJhbmtzIGFuZCBnZW5lcmFsIG1heWhlbSwgb2Z0ZW4gdGFyZ2V0aW5nIENoaWtuIE51Z2dpdCBvciBvdGhlciBjaGFyYWN0ZXJzLiAgSXNjcmVhbSdzIGRlbW9uaWMgYWJpbGl0aWVzIGNvbWUgaW4gaGFuZHkgaGVyZSwgd2hpbGUgRndlbmNoIEZ3eSBjb250cmlidXRlcyB3aXRoIHRoZWlyIGVudGh1c2lhc20gYW5kIHNvbWV0aW1lcy1xdWVzdGlvbmFibGUgaWRlYXMuXFxuXFxuKiAqKkFmZmVjdGlvbmF0ZSBEaXNwbGF5czoqKiAgRGVzcGl0ZSB0aGVpciBjaGFvdGljIG5hdHVyZSwgdGhleSdyZSBjbGVhcmx5IHZlcnkgZm9uZCBvZiBlYWNoIG90aGVyLiBUaGV5IG1pZ2h0IGhvbGQgaGFuZHMgKG9yIHBhd3MpLCBudXp6bGUsIG9yIGV4Y2hhbmdlIGFmZmVjdGlvbmF0ZSBnbGFuY2VzLiAgVGhlc2UgbW9tZW50cyBvZmZlciBhIHN0YXJrIGNvbnRyYXN0IHRvIHRoZWlyIG1vcmUgbWlzY2hpZXZvdXMgYW50aWNzLlxcblxcbiogKipTdXBwb3J0aXZlIENhbWFyYWRlcmllOioqICBUaGV5J3JlIHN1cHBvcnRpdmUgb2YgZWFjaCBvdGhlcidzIGVuZGVhdm9ycywgZXZlbiBpZiB0aG9zZSBlbmRlYXZvcnMgaW52b2x2ZSBjYXVzaW5nIGEgYml0IG9mIHRyb3VibGUuICBUaGV5IG9mdGVuIGVnZyBlYWNoIG90aGVyIG9uLCBidXQgaW4gYSBwbGF5ZnVsIGFuZCBhZmZlY3Rpb25hdGUgd2F5LlxcblxcbiogKipQbGF5ZnVsIEJhbnRlcjoqKiAgVGhleSBlbmdhZ2UgaW4gbGlnaHRoZWFydGVkIHRlYXNpbmcgYW5kIGpva2VzLCBkZW1vbnN0cmF0aW5nIGEgY29tZm9ydGFibGUgYW5kIGZhbWlsaWFyIGR5bmFtaWMuICBUaGlzIGJhbnRlciBjYW4gc29tZXRpbWVzIGJlIGEgYml0IGRhcmssIGdpdmVuIElzY3JlYW0ncyBkZW1vbmljIG5hdHVyZSwgYnV0IGl0IGFsd2F5cyBjb21lcyBhY3Jvc3MgYXMgZ29vZC1uYXR1cmVkLlxcblxcbiogKipTaGFyZWQgSW50ZXJlc3RzOioqIFdoaWxlIHRoZWlyIGludGVyZXN0cyBtaWdodCBzb21ldGltZXMgaW52b2x2ZSBjYXVzaW5nIG1pbm9yIGNoYW9zLCB0aGV5IGFsc28gZW5qb3kgc2ltcGxlIHRoaW5ncyB0b2dldGhlciwgbGlrZSBoYW5naW5nIG91dCB3aXRoIGZyaWVuZHMgb3IgcGFydGljaXBhdGluZyBpbiBhY3Rpdml0aWVzLlxcblxcbkVzc2VudGlhbGx5LCB0aGVpciByZWxhdGlvbnNoaXAgaXMgYSBjb21wbGV4IG1peCBvZiBjaGFvdGljIGVuZXJneSwgZ2VudWluZSBhZmZlY3Rpb24sIGFuZCBzdXBwb3J0aXZlIGNvbXBhbmlvbnNoaXAuICBUaGV5J3JlIHBhcnRuZXJzIGluIGNyaW1lLCBidXQgYWxzbyBwYXJ0bmVycyBpbiBhIG1vcmUgdHJhZGl0aW9uYWwgc2Vuc2UsIGNhcmluZyBmb3IgZWFjaCBvdGhlciBkZXNwaXRlIChvciBwZXJoYXBzIGJlY2F1c2Ugb2YpIHRoZWlyIHNoYXJlZCBsb3ZlIG9mIG1heWhlbS5cXG4ifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "what is chikn nuggit"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "Chikn Nuggit is an animated web series created by Kyra Kupetsky, featuring a quirky, long-eared yellow dog of the same name and his friends, who also have fast-food-inspired names.  The series is known for its comedic, often surreal, and sometimes spooky adventures.  The episodes are short, typically lasting between 15 and 20 seconds each.\\n\\nThe series premiered in October 2020 on platforms like TikTok, X (formerly Twitter), Instagram, and later YouTube.  It quickly gained popularity, amassing over 8 million followers and a billion views across these platforms as of 2023. Chikn Nuggit has also expanded into merchandise.  A longer-form animated series is currently in development as of June 2023, as a collaboration between BuzzFeed Animation Lab and Fredera

## Call `generate_content`

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [7]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name="gemini-1.5-flash-exp-0827")

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    stream=stream,
)

display(Markdown(response.text))

Iscream's chaotic and often unsettling nature adds a unique dynamic to their interactions with Fwench Fwy. It's this duality that makes their relationship so intriguing and unique within the Chikn Nuggit universe. 


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [9]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

what is chikn nuggit 

-------------------------------------------------------------------------------- 

Role: model 

Chikn Nuggit is an animated web series created by Kyra Kupetsky, featuring a quirky, long-eared yellow dog of the same name and his friends, who also have fast-food-inspired names.  The series is known for its comedic, often surreal, and sometimes spooky adventures.  The episodes are short, typically lasting between 15 and 20 seconds each.\n\nThe series premiered in October 2020 on platforms like TikTok, X (formerly Twitter), Instagram, and later YouTube.  It quickly gained popularity, amassing over 8 million followers and a billion views across these platforms as of 2023. Chikn Nuggit has also expanded into merchandise.  A longer-form animated series is currently in development as of June 2023, as a collaboration between BuzzFeed Animation Lab and Frederator Network.\n\nBeyond the humor, Chikn Nuggit also incorporates themes of inclusivity and self-accep